In [1]:
import torch
torch.__version__

'1.3.0'

# 使用PyTorch计算梯度数值

PyTorch的Autograd模块实现了深度学习的算法中的向传播求导数，在张量（Tensor类）上的所有操作，Autograd都能为他们自动提供微分，简化了手动计算导数的复杂过程。

在0.4以前的版本中，Pytorch使用Variable类来自动计算所有的梯度Variable类主要包含三个属性：
data：保存Variable所包含的Tensor；grad：保存data对应的梯度，grad也是个Variable，而不是Tensor，它和data的形状一样；grad_fn：指向一个Function对象，这个Function用来反向传播计算输入的梯度。


从0.4起, Variable 正式合并入Tensor类, 通过Variable嵌套实现的自动微分功能已经整合进入了Tensor类中。虽然为了代码的兼容性还是可以使用Variable(tensor)这种方式进行嵌套, 但是这个操作其实什么都没做。

所以，以后的代码建议直接使用Tensor类进行操作，因为官方文档中已经将Variable设置成过期模块。

要想通过Tensor类本身就支持了使用autograd功能，只需要设置.requries_grad=True

Variable类中的的grad和grad_fn属性已经整合进入了Tensor类中

## Autograd

在张量创建时，通过设置 requires_grad 标识为Ture来告诉Pytorch需要对该张量进行自动求导，PyTorch会记录该张量的每一步操作历史并自动计算

In [20]:
x = torch.rand(5, 5, requires_grad=True)
x

tensor([[0.2600, 0.9090, 0.1332, 0.4449, 0.4841],
        [0.9527, 0.3727, 0.5662, 0.3304, 0.4747],
        [0.1132, 0.7963, 0.6698, 0.4258, 0.9220],
        [0.4497, 0.8236, 0.2788, 0.1540, 0.8647],
        [0.4231, 0.0633, 0.0725, 0.6257, 0.9337]], requires_grad=True)

In [21]:
y = torch.rand(5, 5, requires_grad=True)
y

tensor([[0.8947, 0.7751, 0.2762, 0.5245, 0.6511],
        [0.5652, 0.2597, 0.4120, 0.0223, 0.9939],
        [0.8698, 0.3516, 0.1436, 0.9821, 0.4242],
        [0.2107, 0.2779, 0.8068, 0.0746, 0.7622],
        [0.1967, 0.4408, 0.5445, 0.7797, 0.3747]], requires_grad=True)

PyTorch会自动追踪和记录对与张量的所有操作，当计算完成后调用.backward()方法自动计算梯度并且将计算结果保存到grad属性中。

In [22]:
print(torch.sum(x))
print(torch.sum(y))
z=torch.sum(x+y)
print(z)

tensor(12.5443, grad_fn=<SumBackward0>)
tensor(12.6144, grad_fn=<SumBackward0>)
tensor(25.1587, grad_fn=<SumBackward0>)


在张量进行操作后，grad_fn已经被赋予了一个新的函数，这个函数引用了一个创建了这个Tensor类的Function对象。
Tensor和Function互相连接生成了一个非循环图，它记录并且编码了完整的计算历史。每个张量都有一个.grad_fn属性，如果这个张量是用户手动创建的那么这个张量的grad_fn是None。

下面我们来调用反向传播函数，计算其梯度

## 简单的自动求导

In [23]:
z.backward()
print(x.grad,y.grad)


tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]]) tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])


如果Tensor类表示的是一个标量（即它包含一个元素的张量），则不需要为backward()指定任何参数，但是如果它有更多的元素，则需要指定一个gradient参数，它是形状匹配的张量。
以上的 `z.backward()`相当于是`z.backward(torch.tensor(1.))`的简写。
这种参数常出现在图像分类中的单标签分类，输出一个标量代表图像的标签。

## 复杂的自动求导

In [33]:
x = torch.rand(5, 5, requires_grad=True)
y = torch.rand(5, 5, requires_grad=True)
z= x**2+y**3
z

tensor([[1.3375, 0.4515, 1.2648, 0.4482, 0.5380],
        [0.6818, 0.4639, 0.3706, 0.4531, 0.4299],
        [0.2400, 1.0422, 0.3278, 0.8555, 0.3716],
        [0.0064, 0.7282, 0.6986, 0.5718, 0.4319],
        [0.9419, 0.0083, 0.2887, 0.0401, 0.3131]], grad_fn=<AddBackward0>)

In [34]:
#我们的返回值不是一个标量，所以需要输入一个大小相同的张量作为参数，这里我们用ones_like函数根据x生成一个张量
z.backward(torch.ones_like(x))

print(x.grad)

tensor([[1.5820, 1.3431, 1.7602, 1.1853, 1.4650],
        [1.5869, 1.1457, 1.0872, 1.1979, 0.3615],
        [0.8307, 1.9892, 0.9853, 1.3877, 1.2190],
        [0.1437, 1.7066, 1.2540, 1.4555, 1.2913],
        [0.4694, 0.1039, 0.5578, 0.3937, 0.6403]])


我们可以使用with torch.no_grad()上下文管理器临时禁止对已设置requires_grad=True的张量进行自动求导。这个方法在测试集计算准确率的时候会经常用到，例如：

In [35]:
with torch.no_grad():
    print((x +y*2).requires_grad)

False


使用.no_grad()进行嵌套后，代码不会跟踪历史记录也就是说保存这部分记录的内存不糊会减少内存的使用量并且会加快少许的运算速度。

## Autograd 过程解析

为了说明Pytorch的自动求导原理，我们来尝试分析一下PyTorch的源代码，虽然Pytorch的 Tensor和 TensorBase都是使用CPP来实现的，但是可以使用一些Python的一些方法查看这些对象在Python的属性和状态。
 Python的 `dir()` 返回参数的属性、方法列表。`z`是一个Tensor变量，看看里面有哪些成员变量。

In [36]:
dir(z)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__idiv__',
 '__ilshift__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pow__',
 '__radd__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rfloordiv__',
 '__rmul__',
 '__rpow__',
 '__rshift__',
 '__rsub__',
 '__rtruediv__',
 '__setattr__',
 '__se

返回很多，我们直接排除掉一些Python中特殊方法（以__开头和结束的）和私有方法（以_开头的，直接看几个比较主要的属性：
`.is_leaf`：记录是否是叶子节点。通过这个属性来确定这个变量的类型
在官方文档中所说的“graph leaves”,“leaf variables”，都是指像`x`,`y`这样的手动创建的、而非运算得到的变量，这些变量成为创建变量。
像`z`这样的，是通过计算后得到的结果称为结果变量。

一个变量是创建变量还是结果变量是通过`.is_leaf`来获取的。

In [37]:
print("x.is_leaf="+str(x.is_leaf))
print("z.is_leaf="+str(z.is_leaf))

x.is_leaf=True
z.is_leaf=False


`x`是手动创建的没有通过计算，所以他被认为是一个叶子节点也就是一个创建变量，而`z`是通过`x`与`y`的一系列计算得到的，所以不是叶子结点也就是结果变量。

为什么我们执行`z.backward()`方法会更新`x.grad`和`y.grad`呢？
`.grad_fn`属性记录的就是这部分的操作，虽然`.backward()`方法也是CPP实现的，但是可以通过Python来进行简单的探索。

`grad_fn`：记录并且编码了完整的计算历史

In [11]:
z.grad_fn

`grad_fn`是一个`AddBackward0`类型的变量 `AddBackward0`这个类也是用Cpp来写的,但是我们从名字里就能够大概知道，他是加法(ADD)的反反向传播（Backward），看看里面有些什么东西

In [12]:
dir(z.grad_fn)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_register_hook_dict',
 'metadata',
 'name',
 'next_functions',
 'register_hook',
 'requires_grad']

`next_functions`就是`grad_fn`的精华

In [13]:
z.grad_fn.next_functions

((<PowBackward0 at 0x1208409b0>, 0), (<PowBackward0 at 0x1208408d0>, 0))

`next_functions`是一个tuple of tuple of PowBackward0 and int。

为什么是2个tuple ？
因为我们的操作是`z= x**2+y**3` 刚才的`AddBackward0`是相加，而前面的操作是乘方 `PowBackward0`。tuple第一个元素就是x相关的操作记录

In [14]:
xg = z.grad_fn.next_functions[0][0]
dir(xg)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_register_hook_dict',
 'metadata',
 'name',
 'next_functions',
 'register_hook',
 'requires_grad']

继续深挖

In [15]:
x_leaf=xg.next_functions[0][0]
type(x_leaf)

AccumulateGrad

在PyTorch的反向图计算中，`AccumulateGrad`类型代表的就是叶子节点类型，也就是计算图终止节点。`AccumulateGrad`类中有一个`.variable`属性指向叶子节点。

In [16]:
x_leaf.variable

tensor([[0.1044, 0.6777, 0.2780, 0.5005, 0.4966],
        [0.6328, 0.0611, 0.4004, 0.5564, 0.3631],
        [0.5526, 0.1290, 0.9003, 0.0772, 0.1823],
        [0.9428, 0.6148, 0.9530, 0.4657, 0.0324],
        [0.2976, 0.8095, 0.4215, 0.9606, 0.0161]], requires_grad=True)

这个`.variable`的属性就是我们的生成的变量`x`

In [17]:
print("x_leaf.variable的id:"+str(id(x_leaf.variable)))
print("x的id:"+str(id(x)))

x_leaf.variable的id:4840553424
x的id:4840553424


In [18]:
assert(id(x_leaf.variable)==id(x))

这样整个规程就很清晰了：

1. 当我们执行z.backward()的时候。这个操作将调用z里面的grad_fn这个属性，执行求导的操作。
2. 这个操作将遍历grad_fn的next_functions，然后分别取出里面的Function（AccumulateGrad），执行求导操作。这部分是一个递归的过程直到最后类型为叶子节点。
3. 计算出结果以后，将结果保存到他们对应的variable 这个变量所引用的对象（x和y）的 grad这个属性里面。
4. 求导结束。所有的叶节点的grad变量都得到了相应的更新

最终当我们执行完c.backward()之后，a和b里面的grad值就得到了更新。

## 扩展Autograd
如果需要自定义autograd扩展新的功能，就需要扩展Function类。因为Function使用autograd来计算结果和梯度，并对操作历史进行编码。
在Function类中最主要的方法就是`forward()`和`backward()`他们分别代表了前向传播和反向传播。





一个自定义的Function需要一下三个方法：

    __init__ (optional)：如果这个操作需要额外的参数则需要定义这个Function的构造函数，不需要的话可以忽略。
    
    forward()：执行前向传播的计算代码
    
    backward()：反向传播时梯度计算的代码。 参数的个数和forward返回值的个数一样，每个参数代表传回到此操作的梯度。
        

In [39]:
# 引入Function便于扩展
from torch.autograd.function import Function

In [40]:
# 定义一个乘以常数的操作(输入参数是张量)
# 方法必须是静态方法，所以要加上@staticmethod 
class MulConstant(Function):
    @staticmethod 
    def forward(ctx, tensor, constant):
        # ctx 用来保存信息这里类似self，并且ctx的属性可以在backward中调用
        ctx.constant=constant
        return tensor *constant
    @staticmethod
    def backward(ctx, grad_output):
        # 返回的参数要与输入的参数一样.
        # 第一个输入为3x3的张量，第二个为一个常数
        # 常数的梯度必须是 None.
        return grad_output, None 

定义完我们的新操作后，我们来进行测试

In [64]:
a=torch.rand(3,3,requires_grad=True)
print(a)
b=MulConstant.apply(a,5)
print("a:"+str(a))
print("b:"+str(b)) # b为a的元素乘以5

tensor([[0.4982, 0.2903, 0.2890],
        [0.0957, 0.9245, 0.2022],
        [0.7452, 0.1439, 0.9135]], requires_grad=True)
a:tensor([[0.4982, 0.2903, 0.2890],
        [0.0957, 0.9245, 0.2022],
        [0.7452, 0.1439, 0.9135]], requires_grad=True)
b:tensor([[2.4912, 1.4516, 1.4452],
        [0.4783, 4.6223, 1.0109],
        [3.7262, 0.7196, 4.5675]], grad_fn=<MulConstantBackward>)


反向传播，返回值不是标量，所以`backward`方法需要参数

In [65]:
b.backward(torch.ones_like(a))

In [66]:
a.grad

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

梯度因为1